In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy.linalg import orth

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
sns.set_color_codes()
from time import time

In [3]:
from ridge import *
from ridge_paul import RidgeAlternating

In [4]:
def generate_timings(degree = 3, subspace_dimension = 1, seed = 0, n = 10, N = 1000, which = 'gn', rtol = 1e-5):
    # First define a function of specified degree on a subspace of dimension
    def f(x):
        w = np.ones(x.shape)
        val = np.dot(x.T, w)**degree
        for i in range(subspace_dimension - 1):
            w = np.zeros(x.shape)
            w[i] = 0
            val += np.dot(x, w)**(degree-1)
        return val + 1
    
    np.random.seed(0)
    X = np.random.uniform(size = (N,n))
    fX = np.array([f(x) for x in X])
    
    np.random.seed(seed)
    U0 = orth(np.random.randn(n,subspace_dimension))
    if which == 'gn':
        start_time = time()
        U, c = grassmann_gauss_newton(U0, X, fX, degree = degree, xtol = 0, ftol = 0, gtol = 0, rtol = rtol, disp = False)
        return time() - start_time
        #print fX - np.dot(build_V(U, X, degree), c)
        #return t
    else:
        start_time = time()
        # Paul's code defines the residual as 0.5*np.linalg.norm(f-g)**2
        # so we alter the convergence tolerance to match
        RidgeAlternating(X, fX, U0, degree = degree, tol = 0.5*rtol**2)
        return time() - start_time

In [5]:
n_trials = 10
timing_GN = np.nan*np.ones((6,6, n_trials))
for degree in range(2,6):
    for subspace_dimension in range(1,6):
        for trial in range(n_trials):
            timing_GN[degree, subspace_dimension, trial] = generate_timings(degree, subspace_dimension, seed = trial)
            print 'degree %d, dimension %d, trial %d, time %g' % (degree, subspace_dimension, trial, timing_GN[degree, subspace_dimension, trial])

degree 2, dimension 1, trial 0, time 0.00801706
degree 2, dimension 1, trial 1, time 0.0130949
degree 2, dimension 1, trial 2, time 0.00906706
degree 2, dimension 1, trial 3, time 0.0107789
degree 2, dimension 1, trial 4, time 0.0107918
degree 2, dimension 1, trial 5, time 0.010561
degree 2, dimension 1, trial 6, time 0.0122831
degree 2, dimension 1, trial 7, time 0.012383
degree 2, dimension 1, trial 8, time 0.0122511
degree 2, dimension 1, trial 9, time 0.00801802
degree 2, dimension 2, trial 0, time 0.0133541
degree 2, dimension 2, trial 1, time 0.0135119
degree 2, dimension 2, trial 2, time 0.0167279
degree 2, dimension 2, trial 3, time 0.0168672
degree 2, dimension 2, trial 4, time 0.0201669
degree 2, dimension 2, trial 5, time 0.0208249
degree 2, dimension 2, trial 6, time 0.014807
degree 2, dimension 2, trial 7, time 0.0119562
degree 2, dimension 2, trial 8, time 0.0148509
degree 2, dimension 2, trial 9, time 0.0204999
degree 2, dimension 3, trial 0, time 0.0241699
degree 2, dim

In [6]:
from pgf import PGF
pgf = PGF()
pgf.add('degree', np.arange(2,6))
for dim in range(1,6):
    pgf.add('m%d' % dim, [np.median(timing_GN[d, dim,:]) for d in range(2,6)])
pgf.write('fig_timing_gn.dat')

In [7]:
n_trials = 10
timing_alt = np.nan*np.ones((6,6, n_trials))
for degree in range(2,6):
    for subspace_dimension in range(1,6):
        for trial in range(n_trials):
            timing_alt[degree, subspace_dimension, trial] = generate_timings(degree, subspace_dimension, seed = trial, which = 'paul')
            print 'degree %d, dimension %d, trial %d, time %g' % (degree, subspace_dimension, trial, timing_alt[degree, subspace_dimension, trial])

degree 2, dimension 1, trial 0, time 45.723
degree 2, dimension 1, trial 1, time 82.3834
degree 2, dimension 1, trial 2, time 56.8828
degree 2, dimension 1, trial 3, time 75.37
degree 2, dimension 1, trial 4, time 68.3921
degree 2, dimension 1, trial 5, time 76.503
degree 2, dimension 1, trial 6, time 73.5396
degree 2, dimension 1, trial 7, time 78.7932
degree 2, dimension 1, trial 8, time 80.4574
degree 2, dimension 1, trial 9, time 20.9567
degree 2, dimension 2, trial 0, time 62.8494
degree 2, dimension 2, trial 1, time 14.4923
degree 2, dimension 2, trial 2, time 150.499
degree 2, dimension 2, trial 3, time 158.167
degree 2, dimension 2, trial 4, time 24.9076
degree 2, dimension 2, trial 5, time 12.2499
degree 2, dimension 2, trial 6, time 183.164
degree 2, dimension 2, trial 7, time 153.304
degree 2, dimension 2, trial 8, time 13.1178
degree 2, dimension 2, trial 9, time 197.887
degree 2, dimension 3, trial 0, time 44.0397
degree 2, dimension 3, trial 1, time 192.593
degree 2, dime

In [8]:
from pgf import PGF
pgf = PGF()
pgf.add('degree', np.arange(2,6))
for dim in range(1,6):
    pgf.add('m%d' % dim, [np.median(timing_alt[d, dim,:]) for d in range(2,6)])
pgf.write('fig_timing_alt.dat')